In [1]:
import pandas as pd
df1 = pd.read_csv('./data/cierres_diarios_M1.csv', parse_dates=['Date'], index_col='Date')
df2 = pd.read_csv('./data/cierres_diarios_M2.csv', parse_dates=['Date'], index_col='Date')
df3 = pd.read_csv('./data/cierres_diarios_M3.csv', parse_dates=['Date'], index_col='Date')

In [2]:
# ============================================================
#  ESCENARIO M (window=60, horizon=5)
#  - Usa df1, df2, df3 ya cargados (longitud: 65 filas)
#  - Escalado consistente con el entrenamiento (MinMax a [0,1])
#  - Evalúa SOLO la primera ventana (60→5) de cada df (M1, M2, M3)
# ============================================================

# Eliminación de nulos
df1.ffill(inplace=True)
df1.bfill(inplace=True)
df2.ffill(inplace=True)
df2.bfill(inplace=True)
df3.ffill(inplace=True)
df3.bfill(inplace=True)

import pandas as pd
import numpy as np
import tensorflow as tf
from joblib import load
from typing import List

# Cargar scaler y columnas base (mismo orden que en entrenamiento)
scaler = load("scaler_modelos.joblib")

# Como el scaler tiene 'feature_names_in_', usamos ese orden:
feature_order = list(scaler.feature_names_in_)

# Filtramos/ordenamos exactamente como en entrenamiento
df1 = df1[feature_order].copy()
df2 = df2[feature_order].copy()
df3 = df3[feature_order].copy()

# Escalar con el mismo scaler
df1_scaled = pd.DataFrame(
    scaler.transform(df1[feature_order]),
    index=df1.index,
    columns=feature_order
).astype("float32")

df2_scaled = pd.DataFrame(
    scaler.transform(df2[feature_order]),
    index=df2.index,
    columns=feature_order
).astype("float32")

df3_scaled = pd.DataFrame(
    scaler.transform(df3[feature_order]),
    index=df3.index,
    columns=feature_order
).astype("float32")

# Carga del modelo M (60→5)
from tensorflow.keras.models import load_model
path_m = "./models_gru_huber_sweep/gru_huber_w60_h5_delta0_1_M.keras"
model_m = load_model(path_m, compile=False)
delta_m = 0.1

# === Datasets coherentes con el entrenamiento: usar SIEMPRE datos ESCALADOS ===
def make_dataset(data_scaled, window_size, horizon, batch_size=32, shuffle=False):
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data_scaled.values,
        targets=None,
        sequence_length=window_size + horizon,
        sequence_stride=1,
        batch_size=batch_size,
        shuffle=shuffle
    )
    return ds.map(
        lambda seq: (
            tf.cast(seq[:, :window_size, :], tf.float32),   # X
            tf.cast(seq[:, window_size:, :], tf.float32)    # y
        )
    )

window_size = 60
horizon = 5

# Evaluar SOLO la primera (60→5) del intervalo (65 filas: 60 + 5)
ds1 = make_dataset(df1_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds2 = make_dataset(df2_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds3 = make_dataset(df3_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)

# === Métricas y pérdidas ===
def _eps_tag(x: float) -> str:
    return str(x).replace('.', '_')

def log_cosh_metric(y_true, y_pred):
    e = tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32)
    # logcosh(x) = |x| + softplus(-2|x|) - log(2)  → estable y sin overflow
    ae = tf.abs(e)
    return tf.reduce_mean(ae + tf.nn.softplus(-2.0 * ae) - tf.math.log(2.0))
log_cosh_metric.__name__ = "log_cosh"

def make_huber_loss(delta: float):
    base = tf.keras.losses.Huber(delta=float(delta))
    def huber_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32); y_pred = tf.cast(y_pred, tf.float32)
        return base(y_true, y_pred)
    huber_loss.__name__ = f"huber_delta_{_eps_tag(delta)}"
    return huber_loss

def make_within_eps_vector_metric(eps_vec: np.ndarray, tag: str):
    """eps_vec shape (F,) en la MISMA escala que y_true/y_pred (normalizada)."""
    eps_tf = tf.constant(eps_vec.astype(np.float32), dtype=tf.float32)  # (F,)
    def within_eps(y_true, y_pred):
        diff = tf.abs(tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32))  # (B,H,F)
        thr  = eps_tf[tf.newaxis, tf.newaxis, :]                                   # (1,1,F)
        hit  = tf.cast(diff <= thr, tf.float32)
        return tf.reduce_mean(hit)
    within_eps.__name__ = tag
    return within_eps

def build_within_metrics_minmax(scaler, eps_list: List[float], n_features: int):
    """
    MinMaxScaler: eps = porcentaje * data_range_ por feature.
    OJO: y_true/y_pred están normalizados, así que el umbral ya se pasa NORMALIZADO.
    Con MinMax a [0,1], 'porcentaje del rango' == el propio porcentaje.
    Para mantener la semántica, se usa directamente eps_list (0.5%, 1%, ... del rango).
    """
    metrics = [log_cosh_metric]

    # Verificación rápida de consistencia del scaler
    if not hasattr(scaler, "data_range_"):
        raise ValueError("Se esperaba un MinMaxScaler con atributo data_range_.")
    if len(scaler.data_range_) != n_features:
        raise ValueError("scaler.data_range_ no coincide con n_features.")

    # En escala normalizada [0,1], el 'porcentaje del rango' es exactamente el valor eps.
    for e in eps_list:
        eps_vec = np.full((n_features,), float(e), dtype=np.float32)
        tag = f"within_eps_{_eps_tag(e)}"
        metrics.append(make_within_eps_vector_metric(eps_vec, tag))
    return metrics

def compute_autc_from_results(res: dict, eps_list: List[float]) -> float:
    """
    Calcula AUTC normalizando por el rango [ε_min, ε_max], ∈ [0,1].
    Toma los valores within_eps_* devueltos por model.evaluate(return_dict=True).
    """
    eps = np.array(sorted(eps_list), dtype=np.float32)
    acc = np.array([res.get(f"within_eps_{str(e).replace('.','_')}", np.nan) for e in eps],
                   dtype=np.float32)

    mask = np.isfinite(acc)
    if mask.sum() < 2:    # Si faltan puntos o hay NaN, integra sobre los disponibles (requiere ≥ 2 puntos).
        return float("nan")

    eps = eps[mask]
    acc = acc[mask]
    return float(np.trapz(acc, eps) / (eps[-1] - eps[0]))

# === Compilar el modelo con las métricas within-ε correctas (escala normalizada) ===
EPS_LIST = [0.005, 0.01, 0.02, 0.05, 0.1]
n_features = len(feature_order)

metrics = build_within_metrics_minmax(scaler, EPS_LIST, n_features=n_features)
model_m.compile(optimizer="adam", loss=make_huber_loss(delta_m), metrics=metrics)

def evaluate_and_print(tag, dataset):
    res = model_m.evaluate(dataset, return_dict=True, verbose=0)
    print(f"\n=== Resultados {tag} (60→5) ===")
    print(f"  Huber (loss):              {res.get('loss', float('nan')):.6f}")
    print(f"  log_cosh:                  {res.get('log_cosh', float('nan')):.6f}")
    for e in EPS_LIST:
        key = f"within_eps_{str(e).replace('.','_')}"
        print(f"  {key:26s}: {res.get(key, float('nan')):.6f}")
    autc = compute_autc_from_results(res, EPS_LIST)
    print(f"  AUTC[{min(EPS_LIST):.3f}–{max(EPS_LIST):.3f}]:         {autc:.6f}")
    return res

# === Evaluación de M1, M2 y M3 (primera ventana 60→5 de cada df) ===
res1 = evaluate_and_print("M1", ds1)
res2 = evaluate_and_print("M2", ds2)
res3 = evaluate_and_print("M3", ds3)

# === Ventana ÚNICA (60→5) por cada M para inspección puntual (si lo necesitas luego) ===
def single_window_arrays(df_scaled, w=60, h=5):
    if len(df_scaled) < w + h:
        raise ValueError("No hay suficientes días hábiles para 60→5.")
    X = df_scaled.iloc[0:w].to_numpy()[np.newaxis, :, :]              # (1,w,F)
    Y = df_scaled.iloc[w:w+h].to_numpy()[np.newaxis, :, :]            # (1,h,F)
    return X.astype("float32"), Y.astype("float32")


2025-09-21 19:58:57.605019: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-21 19:58:57.605036: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-21 19:58:57.605041: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-21 19:58:57.605057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-21 19:58:57.605069: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-21 19:58:58.027582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



=== Resultados M1 (60→5) ===
  Huber (loss):              0.327739
  log_cosh:                  2.903260
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.008000
  within_eps_0_02           : 0.016000
  within_eps_0_05           : 0.024000
  within_eps_0_1            : 0.112000
  AUTC[0.005–0.100]:         0.043579

=== Resultados M2 (60→5) ===
  Huber (loss):              0.302091
  log_cosh:                  2.624135
  within_eps_0_005          : 0.016000
  within_eps_0_01           : 0.024000
  within_eps_0_02           : 0.032000
  within_eps_0_05           : 0.056000
  within_eps_0_1            : 0.088000
  AUTC[0.005–0.100]:         0.055789

=== Resultados M3 (60→5) ===
  Huber (loss):              0.287891
  log_cosh:                  2.447360
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.000000
  within_eps_0_02           : 0.000000
  within_eps_0_05           : 0.008000
  within_eps_0_1            : 0.072000
  AUTC[0.005–0.100]: 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1) Construir tabla con todas las métricas por escenario ---
def res_to_series(tag, res, eps_list):
    d = {
        "loss": res.get("loss", np.nan),
        "log_cosh": res.get("log_cosh", np.nan),
        "AUTC": compute_autc_from_results(res, eps_list),
    }
    for e in eps_list:
        key = f"within_eps_{str(e).replace('.','_')}"
        d[key] = res.get(key, np.nan)
    s = pd.Series(d, name=tag)
    return s

m1 = res_to_series("M1", res1, EPS_LIST)
m2 = res_to_series("M2", res2, EPS_LIST)
m3 = res_to_series("M3", res3, EPS_LIST)

df_all = pd.concat([m1, m2, m3], axis=1).T  # filas: escenarios; cols: métricas
df_mean = df_all.mean(axis=0)

print("\n==== Tabla de métricas por escenario ====")
print(df_all.round(6))
print("\n==== Medias (M1,M2,M3) ====")
print(df_mean.round(6))


==== Tabla de métricas por escenario ====
        loss  log_cosh      AUTC  within_eps_0_005  within_eps_0_01  \
M1  0.327739  2.903260  0.043579             0.000            0.008   
M2  0.302091  2.624135  0.055789             0.016            0.024   
M3  0.287891  2.447360  0.022316             0.000            0.000   

    within_eps_0_02  within_eps_0_05  within_eps_0_1  
M1            0.016            0.024           0.112  
M2            0.032            0.056           0.088  
M3            0.000            0.008           0.072  

==== Medias (M1,M2,M3) ====
loss                0.305907
log_cosh            2.658252
AUTC                0.040561
within_eps_0_005    0.005333
within_eps_0_01     0.010667
within_eps_0_02     0.016000
within_eps_0_05     0.029333
within_eps_0_1      0.090667
dtype: float64


In [ ]:
# --- 2) Gráfica A: curvas within-ε (ε en eje X, accuracy en eje Y) ---
eps = np.array(EPS_LIST, dtype=float)

def pick_from_df(df_row, eps_list):
    vals = []
    for e in eps_list:
        key = f"within_eps_{str(e).replace('.','_')}"
        vals.append(df_row[key])
    return np.array(vals, dtype=float)

w_m1 = pick_from_df(m1, EPS_LIST)
w_m2 = pick_from_df(m2, EPS_LIST)
w_m3 = pick_from_df(m3, EPS_LIST)

plt.figure()
plt.plot(eps, w_m1, marker="o", label="M1")
plt.plot(eps, w_m2, marker="o", label="M2")
plt.plot(eps, w_m3, marker="o", label="M3")
plt.title("Curva within-ε por escenario (60→5)")
plt.xlabel("ε (escala MinMax)")
plt.ylabel("Proporción de aciertos dentro de ε")
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# --- 3) Gráfica B: barras para loss, log_cosh y AUTC ---
labels = ["M1","M2","M3"]
loss_vals = [m1["loss"], m2["loss"], m3["loss"]]
lc_vals   = [m1["log_cosh"], m2["log_cosh"], m3["log_cosh"]]
autc_vals = [m1["AUTC"], m2["AUTC"], m3["AUTC"]]

# Barras: loss
plt.figure()
plt.bar(labels, loss_vals)
plt.title("Huber loss por escenario")
plt.ylabel("loss")
plt.tight_layout()
plt.show()

# Barras: log_cosh
plt.figure()
plt.bar(labels, lc_vals)
plt.title("log_cosh por escenario")
plt.ylabel("log_cosh")
plt.tight_layout()
plt.show()

# Barras: AUTC
plt.figure()
plt.bar(labels, autc_vals)
plt.title(f"AUTC[{min(EPS_LIST):.3f}–{max(EPS_LIST):.3f}] por escenario")
plt.ylabel("AUTC")
plt.tight_layout()
plt.show()

In [ ]:
# --- 4) (Opcional) imprimir medias “bonitas” ya redondeadas ---
print("\n==== Medias (redondeadas) ====")
for k, v in df_mean.items():
    print(f"{k:>20s}: {v:.6f}")